## 14년

In [ ]:
small_years =  [14]

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import copy
import json
import random
import io
import distutils.dir_util
import warnings
warnings.filterwarnings("ignore")
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity

import os, sys, gc, time, warnings, pickle, psutil, random
import gc
import time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

        
from collections import Counter
from tqdm import tqdm

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)

    return json_obj

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath(parent)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [ ]:
train = pd.read_parquet('/kaggle/input/song-traim-song-tag-dict-tag-song-dict/train(song_trim).parquet')




In [ ]:
val = pd.read_json('/kaggle/input/song-traim-song-tag-dict-tag-song-dict/test(lower).json')

In [ ]:
def most_popular(playlists, col, topk_count):
    c = Counter()

    for doc in playlists[col]:
        c.update(doc)

    topk = c.most_common(topk_count)
    return c, [k for k, v in topk]

def remove_seen(seen, l):
    seen = set(seen)
    ## 데이터에 l의 원소가 없으면 l의 원소를 채워 넣는다. 
    return [x for x in l if not (x in seen)]

In [ ]:
def check_target_type(val):
    no_song = []
    no_both = []
    no_tag = []
    yes_index = []

    for i in range(len(val)):
        if (not val.loc[i:i, 'songs'].values[0])&(not not val.loc[i:i, 'tags'].values[0]): ## 노래는 없는데 태그는 있어
            no_song.append(i)

        elif (not not val.loc[i:i, 'songs'].values[0]) & (not val.loc[i:i, 'tags'].values[0]): ## 노래는 있는데 태그가 없으
            no_tag.append(i)  

        elif (not val.loc[i:i, 'songs'].values[0]) & (not val.loc[i:i, 'tags'].values[0]): ## 노래도 없고 태그도 없어
            no_both.append(i)          

        else: ## 둘다 있음, 아마도 노래 몇 곡이 누락 됐어 
            yes_index.append(i)
            
    print("노래가 하나도 없음 : ", len(no_song))
    print("노래, 태그 둘 다 존재함 : ", len(yes_index))
    print("태그가 없음 : ",len(no_tag) )
    print("둘다 없음 : ", len(no_both))
    
    return no_song, no_both, no_tag, yes_index
        

In [ ]:
no_song, no_both, no_tag, yes_index = check_target_type(val)

In [ ]:
v1v3_index= yes_index+ no_tag


train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))
train['date'] = train['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )

val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))
val['date'] = val['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )


year_list = list(train['year'].unique())
year_list= sorted(year_list)

v1v3 = val[val.index.isin(v1v3_index)]


val_tmp = v1v3[v1v3['year'].isin(small_years)]
print("val year shape", val_tmp.shape)
val_tmp.reset_index(inplace = True, drop = True)

In [ ]:
date_list = list(train['date'].unique())
date_list= sorted(date_list)

date_dict = {}
for i,k in enumerate(date_list):
    date_dict[k] = i

popular_date_tag = []
for i in range(len(date_list)):
    if i <= 13:  ## 13이전에는 한 달에 100개가 안나옴
        year = int(str(date_list[i])[0]) ## year
        tmp = train[train['year'] == year]
        _, year_tag = most_popular(tmp, "tags", 100)
        popular_date_tag.append({
            "date" : date_list[i],
            "tags" : year_tag,})
        
    else:
        tmp = train[train['date'] == date_list[i]]
        _, date_tag = most_popular(tmp, "tags", 100)
        popular_date_tag.append({
            "date" : date_list[i],
            "tags" : date_tag,})  
        
        
popular_date_song = []
for i in range(len(date_list)):
    if i <= 9:  ## 11이전에는 한 달에 100개가 안나옴
        year = int(str(date_list[i])[0]) ## year
        tmp = train[train['year'] == year]
        _, date_song = most_popular(tmp, "songs", 200)
        popular_date_song.append({
            "date" : date_list[i],
            "songs" : date_song,})               
        
    else:
        tmp = train[train['date'] == date_list[i]]
        _, date_song = most_popular(tmp, "songs", 200)
        popular_date_song.append({
            "date" : date_list[i],
            "songs" : date_song,})          

In [ ]:
tmp = train[(train['year'].isin(small_years))]
like_num = int(np.percentile(tmp['like_cnt'], 70))
tmp = tmp[(tmp['like_cnt'] >like_num)]
tmp.reset_index(inplace = True, drop = True)
print("train best playlist", tmp.shape)
print('train+valid = ', tmp.shape[0]+val_tmp.shape[0])

In [ ]:
song_tag_dict = load_json('/kaggle/input/song-traim-song-tag-dict-tag-song-dict/song_tag_dict.json')

In [ ]:
def before_pivot(data):
    result = []
    for i in tqdm(range(len(data))):
        for song in data.loc[i:i, 'songs'].values[0]:
            df = pd.DataFrame({
                        'id' : [data.loc[i:i, 'id'].values[0]],
                        'song' : [song],
                        'point' : 1
                    })
            result += [df]

    final_result = pd.concat(result)
    print(final_result.shape)
    return final_result

In [ ]:
before_tmp = before_pivot(tmp)

## train만 1보다 작은 노래는 없애준다.
tmp_dict = before_tmp['song'].value_counts().to_dict()
before_tmp['count']= before_tmp['song'].apply(lambda x : tmp_dict[x])
before_tmp = before_tmp[before_tmp['count'] > 1]
print(before_tmp.shape)

In [ ]:
before_val_tmp = before_pivot(val_tmp)

In [ ]:
final_result = pd.concat([before_tmp, before_val_tmp])
del before_tmp, before_val_tmp
gc.collect()

In [ ]:
def make_cosine_predict(final_result):
    R_df = final_result.pivot(index = 'id', columns ='song', values = 'point').fillna(0)
    del final_result
    
    print("make cosine data")    
    cosine_array = cosine_similarity(R_df, R_df)
    predicted_array = np.zeros(shape=(len(R_df.index),len(R_df.columns))) 
    for i in tqdm(range(len(cosine_array))):
        top_200 = cosine_array[i].argsort()[-201:][::-1]
        top_200 = np.delete(top_200, 0)

        weighted_sum = np.array([0])
        for top_idx in top_200:
            weighted_sum = weighted_sum + (cosine_array[i][top_idx] * R_df.values[top_idx])
        predicted = weighted_sum / len(top_200)
        predicted_array[i] = predicted
    iu_predicted = R_df.values*(-99999) + predicted_array
    
    print("make dic data")    
    cf_dic = {}
    for i in range(len(iu_predicted)):
        cf_dic[R_df.index[i]] = R_df.columns[iu_predicted[i].argsort()[-200:][::-1]].tolist()
    
    return cf_dic

In [ ]:
cf_dic = make_cosine_predict(final_result)

In [ ]:
val_tmp.reset_index(inplace = True, drop = True)


In [ ]:
val_tmp.head(2)

In [ ]:
val1_predict = []
for i in range(len(val_tmp)):
    ############### song 채우기 ##################################
    date = date_dict[val_tmp.loc[i:i, 'date'].values[0]]
    row_number = val_tmp.loc[i:i, 'id'].values[0]
    cur_song = list(cf_dic[row_number])
       
    #############################################################
    
    
    ############### tag 채우기 ##################################
    tag_counter = Counter()
    for song in list(val_tmp.loc[i:i, 'songs'].values[0]):
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key= lambda x:x[1], reverse = True)        
    cur_tag = []
    for k in tag_counter[:10]:
        cur_tag.append(k[0])     
    if (len(cur_tag) < 10) & (len(cur_tag) > 0 ):
        ## 부족하면 채워준다.
        update_tag = remove_seen(cur_tag, popular_date_tag[date]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:10]
    if len(cur_tag) == 0:
        cur_tag = popular_date_tag[date]['tags'][:10]
        
    #############################################################    
    
    
    val1_predict.append({
        "id" : list(val_tmp.loc[i:i, 'id'])[0],
        "songs": remove_seen(list(val_tmp.loc[i:i, 'songs'])[0], cur_song)[:100],
        "tags": cur_tag,
    })
    



In [ ]:
## save file
write_json(val1_predict, 'test1+3_'+str(small_years)+'.json')


In [ ]:

# start = time.time()  # 시작 시간 저장     
# data = pd.concat([tmp, val_tmp])
# data.reset_index(inplace = True, drop = True)

# id2int = {u:i for i, u in enumerate((data['id']))}


# print("merge data", data.shape)

# print('-------------------------')


# ## 모든 point는 1임, 나중에
# result = []
# for i in range(len(data)):
#     for song in data.loc[i:i, 'songs'].values[0]:
#         df = pd.DataFrame({
#                     'id' : [data.loc[i:i, 'id'].values[0]],
#                     'song' : [song],
#                     'point' : 1
#                 })
#         result += [df]

# final_result = pd.concat(result)

# del result, tmp, data, val_tmp
# gc.collect()



# tmp_dict = final_result['song'].value_counts().to_dict()

# final_result['count']= final_result['song'].apply(lambda x : tmp_dict[x])
# final_result = final_result[final_result['count'] > 1]


# R_df = final_result.pivot(index = 'id', columns ='song', values = 'point').fillna(0)
# del final_result

# print("make cosine data")    
# cosine_array = cosine_similarity(R_df, R_df)
# predicted_array = np.zeros(shape=(len(R_df.index),len(R_df.columns))) 
# for i in tqdm(range(len(cosine_array))):
#     top_200 = cosine_array[i].argsort()[-201:][::-1]
#     top_200 = np.delete(top_200, 0)

#     weighted_sum = np.array([0])
#     for top_idx in top_200:
#         weighted_sum = weighted_sum + (cosine_array[i][top_idx] * R_df.values[top_idx])
#     predicted = weighted_sum / len(top_200)
#     predicted_array[i] = predicted
# iu_predicted = R_df.values*(-99999) + predicted_array

# print("make dic data")    
# cf_dic = {}
# for i in range(len(iu_predicted)):
#     cf_dic[R_df.index[i]] = R_df.columns[iu_predicted[i].argsort()[-200:][::-1]].tolist()
